In [33]:
import pandas as pd
data=pd.read_csv('all_kindle_review.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [34]:
data=data[["reviewText","rating"]]

In [35]:
data.shape

(12000, 2)

In [36]:
data.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [37]:
data["rating"].unique()

array([3, 5, 4, 2, 1])

In [38]:
data["rating"].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [39]:
#Preprocessing and Cleaning the text
## postive review is 1 and negative review is 0
data['rating']=data['rating'].apply(lambda x:0 if x<3 else 1)

In [40]:
data["rating"].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [41]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [42]:
data["reviewText"]=data["reviewText"].str.lower()

In [43]:
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [44]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rahulmanchanda/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
from bs4 import BeautifulSoup

In [49]:
## Removing special characters
data['reviewText']=data['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
data['reviewText']=data['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
data['reviewText']=data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
data['reviewText']=data['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
## Remove any additional spaces
data['reviewText']=data['reviewText'].apply(lambda x: " ".join(x.split()))

In [51]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [53]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [57]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [58]:
data['reviewText']=data['reviewText'].apply(lambda x:lemmatize_words(x))

In [60]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data["reviewText"], data["rating"], test_size=0.20, random_state=42)

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [64]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [65]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [66]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [67]:
y_pred_bow=nb_model_bow.predict(X_test_bow)
y_pred_tfidf=nb_model_tfidf.predict(X_test_tfidf)

In [68]:
confusion_matrix(y_test,y_pred_bow)

array([[499, 304],
       [717, 880]])

In [69]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.5745833333333333


In [70]:
confusion_matrix(y_test,y_pred_tfidf)

array([[488, 315],
       [696, 901]])

In [71]:
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.57875


In [72]:
from gensim.models import Word2Vec

# Train Word2Vec model
model = Word2Vec(sentences=data['reviewText'], vector_size=100, window=5, min_count=2, workers=4)


In [75]:
import numpy as np

def avg_word2vec(tokens, model):
    valid_words = [w for w in tokens if w in model.wv.index_to_key]
    if not valid_words:  # if review has no known words
        return np.zeros(model.vector_size)
    return np.mean(model.wv[valid_words], axis=0)

X = np.array([avg_word2vec(tokens, model) for tokens in data['reviewText']])
y = data['rating'].values


In [76]:
from sklearn.model_selection import train_test_split

X_train_avgword2vec, X_test_avgword2vec, y_train_avgword2vec, y_test_avgword2vec = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)


In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_avgword2vec, y_train_avgword2vec)

y_pred = clf.predict(X_test_avgword2vec)

print("Accuracy:", accuracy_score(y_test_avgword2vec, y_pred))
print(classification_report(y_test_avgword2vec, y_pred))


Accuracy: 0.668
              precision    recall  f1-score   support

           0       0.56      0.02      0.04      1000
           1       0.67      0.99      0.80      2000

    accuracy                           0.67      3000
   macro avg       0.61      0.51      0.42      3000
weighted avg       0.63      0.67      0.55      3000



In [81]:
confusion_matrix(y_test_avgword2vec, y_pred)

array([[  20,  980],
       [  16, 1984]])

In [83]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("punkt")

stop_words = set(stopwords.words("english"))

def clean_text(text):
    # Lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove punctuation/numbers
    text = re.sub(r"[^a-z\s]", '', text)
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    tokens = [w for w in tokens if w not in stop_words and len(w) > 2]
    return tokens


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rahulmanchanda/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rahulmanchanda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [85]:
def predict_sentiment(review):
    tokens = clean_text(review)
    vec = avg_word2vec(tokens, model).reshape(1, -1)
    return clf.predict(vec)[0]

print(predict_sentiment("This book was absolutely amazing, I loved it!"))



1
